# Learning GeoPandas

Here is one way of getting the solution. There are others, feel free to
share yours.

``` py
import geopandas as gpd
import contextily as ctx
```

``` py
price = gpd.read_file("https://opendata.iprpraha.cz/CUR/SED/SED_CenovaMapa_p/S_JTSK/SED_CenovaMapa_p_shp.zip")
districts = gpd.read_file("https://opendata.iprpraha.cz/CUR/MAP/MAP_MESTSKECASTI_P/S_JTSK/MAP_MESTSKECASTI_P_shp.zip")
```

``` py
price["CENA"] = price["CENA"].replace("N", None).astype('float')
```

-   Plot the price.
-   Plot boundaries on top. Try different colours to get a nice
    combination of colours.
-   Show a legend.
-   Use CartoDB Voyager or CartoDB Dark Matter basemap.
-   Can you figure out how to change the colormap?
-   Can you change the transparency of polygons?

Interactive:

``` py
m = price.explore("CENA", legend=True, tiles="CartoDB Voyager", cmap="plasma", style_kwds={"fillOpacity": .5})
districts.boundary.explore(m=m, color="red")
```

Static:

``` py
ax = price.plot("CENA", legend=True, cmap="plasma", alpha=.5)
districts.boundary.plot(ax=ax, color="red")
ctx.add_basemap(ax=ax, crs=price.crs, source="CartoDB Voyager")
```

Create a convex hull around each polygon in price.

``` py
price["hull"] = price.convex_hull
```

Calculate the area of these convex hulls.

``` py
price["hull_area"] = price["hull"].area
```

Find the 20% smallest convex hulls in the GeoDataFrame. Create a new
object (e.g. smallest) only with them.

``` py
total_length = len(price)  # get total length
twenty_percent = total_length / 5  # get the length of 20%
twenty_percent
```

You can either type it in:

``` py
smallest = price.sort_values("hull_area").iloc[:1889]
```

Or convert the `float` to `int` and use it programmatically.

``` py
smallest = price.sort_values("hull_area").iloc[:int(twenty_percent)]
```

Create a multi-layer map of Prague where the smallest areas are coloured
in one colour, and the rest appear in black. *(I need to revise this
task, the result is not great…)*

``` py
ax = price.plot(color="black")
smallest.plot(ax=ax, color="red")
```

Join the two GeoDataFrame using .sjoin() or .overlay() methods.

``` py
price_w_district = price.sjoin(districts)
```

Is the mean price higher in Praha 3 or Praha 6?

``` py
sorted_price = price_w_district.groupby("NAZEV_MC")["CENA"].mean().sort_values()
sorted_price
```

From the Series above, you can read that Praha 3 is more expensive than
Praha 6.

Which district is the cheapest?

Again, you can read that it is Praha-Přední Kopanina. But if you want to
get that programmatically, you will need to access the index.

``` py
sorted_price.idxmin()
```

What is the difference between the cheapest and the most expensive one?

``` py
sorted_price.max() - sorted_price.min()
```